In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

from torch.autograd import Variable
from torchvision import datasets, models, transforms

In [2]:
import numpy as np
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [53]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [5]:
data_dir = 'data'

In [23]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

In [7]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'val']}

In [8]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
dataset_sizes

{'train': 4279, 'val': 471}

In [9]:
class_names = image_datasets['train'].classes
class_names

['Black-grass',
 'Charlock',
 'Cleavers',
 'Common Chickweed',
 'Common wheat',
 'Fat Hen',
 'Loose Silky-bent',
 'Maize',
 'Scentless Mayweed',
 'Shepherds Purse',
 'Small-flowered Cranesbill',
 'Sugar beet']

In [10]:
model = models.resnet34(pretrained=True)

In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [13]:
def train(model, dataloader, optimizer, criterion, epoch):
    model.train()
    
    for batch_idx, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        prediction = model(x)

        loss = criterion(prediction, y)
        loss.backward()

        optimizer.step() 
        
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), loss.data.item()))

In [14]:
def val(model, dataloader, criterion):
    model.eval()
    
    val_loss = 0
    correct = 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            
            prediction = model(x)
            val_loss += criterion(prediction, y).item()
            
            y_hat = prediction.max(1, keepdim=True)[1]
            correct += y_hat.eq(y.view_as(y_hat)).sum().item()
    
    val_loss /= len(dataloader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(dataloader.dataset),
        100. * correct / len(dataloader.dataset)))

In [15]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model.to(device);

In [16]:
for epoch in range(1, 50+1):
    train(model, dataloaders['train'], optimizer, criterion, epoch)
    val(model, dataloaders['val'], criterion)
    for param_group in optimizer.param_groups:
        param_group['lr'] *= 0.9

Train Epoch: 1 [0/4279 (0%)]	Loss: 2.744014
Train Epoch: 1 [640/4279 (15%)]	Loss: 2.297440
Train Epoch: 1 [1280/4279 (30%)]	Loss: 2.286605
Train Epoch: 1 [1920/4279 (45%)]	Loss: 2.371330
Train Epoch: 1 [2560/4279 (60%)]	Loss: 2.446955
Train Epoch: 1 [3200/4279 (75%)]	Loss: 2.383301
Train Epoch: 1 [3840/4279 (90%)]	Loss: 1.865659

Test set: Average loss: 0.0626, Accuracy: 177/471 (38%)

Train Epoch: 2 [0/4279 (0%)]	Loss: 1.745012
Train Epoch: 2 [640/4279 (15%)]	Loss: 1.877338
Train Epoch: 2 [1280/4279 (30%)]	Loss: 1.835618
Train Epoch: 2 [1920/4279 (45%)]	Loss: 1.365567
Train Epoch: 2 [2560/4279 (60%)]	Loss: 1.360915
Train Epoch: 2 [3200/4279 (75%)]	Loss: 1.430604
Train Epoch: 2 [3840/4279 (90%)]	Loss: 1.588620

Test set: Average loss: 0.0543, Accuracy: 198/471 (42%)

Train Epoch: 3 [0/4279 (0%)]	Loss: 1.510772
Train Epoch: 3 [640/4279 (15%)]	Loss: 1.409537
Train Epoch: 3 [1280/4279 (30%)]	Loss: 1.340947
Train Epoch: 3 [1920/4279 (45%)]	Loss: 1.329670
Train Epoch: 3 [2560/4279 (60%)]	Lo


Test set: Average loss: 0.0076, Accuracy: 438/471 (93%)

Train Epoch: 22 [0/4279 (0%)]	Loss: 0.366831
Train Epoch: 22 [640/4279 (15%)]	Loss: 0.741157
Train Epoch: 22 [1280/4279 (30%)]	Loss: 0.175128
Train Epoch: 22 [1920/4279 (45%)]	Loss: 0.097906
Train Epoch: 22 [2560/4279 (60%)]	Loss: 0.336926
Train Epoch: 22 [3200/4279 (75%)]	Loss: 0.223252
Train Epoch: 22 [3840/4279 (90%)]	Loss: 0.221830

Test set: Average loss: 0.0087, Accuracy: 430/471 (91%)

Train Epoch: 23 [0/4279 (0%)]	Loss: 0.423840
Train Epoch: 23 [640/4279 (15%)]	Loss: 0.365186
Train Epoch: 23 [1280/4279 (30%)]	Loss: 0.125583
Train Epoch: 23 [1920/4279 (45%)]	Loss: 0.248596
Train Epoch: 23 [2560/4279 (60%)]	Loss: 0.295593
Train Epoch: 23 [3200/4279 (75%)]	Loss: 0.161696
Train Epoch: 23 [3840/4279 (90%)]	Loss: 0.061307

Test set: Average loss: 0.0062, Accuracy: 447/471 (95%)

Train Epoch: 24 [0/4279 (0%)]	Loss: 0.429714
Train Epoch: 24 [640/4279 (15%)]	Loss: 0.607943
Train Epoch: 24 [1280/4279 (30%)]	Loss: 0.434352
Train Ep

Train Epoch: 42 [3200/4279 (75%)]	Loss: 0.107244
Train Epoch: 42 [3840/4279 (90%)]	Loss: 0.243459

Test set: Average loss: 0.0050, Accuracy: 459/471 (97%)

Train Epoch: 43 [0/4279 (0%)]	Loss: 0.463927
Train Epoch: 43 [640/4279 (15%)]	Loss: 0.398275
Train Epoch: 43 [1280/4279 (30%)]	Loss: 0.065818
Train Epoch: 43 [1920/4279 (45%)]	Loss: 0.557654
Train Epoch: 43 [2560/4279 (60%)]	Loss: 0.127402
Train Epoch: 43 [3200/4279 (75%)]	Loss: 0.163804
Train Epoch: 43 [3840/4279 (90%)]	Loss: 0.179074

Test set: Average loss: 0.0048, Accuracy: 456/471 (97%)

Train Epoch: 44 [0/4279 (0%)]	Loss: 0.190772
Train Epoch: 44 [640/4279 (15%)]	Loss: 0.229387
Train Epoch: 44 [1280/4279 (30%)]	Loss: 0.188438
Train Epoch: 44 [1920/4279 (45%)]	Loss: 0.062664
Train Epoch: 44 [2560/4279 (60%)]	Loss: 0.205014
Train Epoch: 44 [3200/4279 (75%)]	Loss: 0.225182
Train Epoch: 44 [3840/4279 (90%)]	Loss: 0.138504

Test set: Average loss: 0.0051, Accuracy: 456/471 (97%)

Train Epoch: 45 [0/4279 (0%)]	Loss: 0.144741
Train E

In [17]:
torch.save(model, 'models/resnet34-50.pt')

In [27]:
test_folder = !ls data/test

In [30]:
from PIL import Image

In [54]:
result = []
for test_image_file in test_folder:
    test_image = Image.open(f'data/test/{test_image_file}')
    test_image_tensor = data_transforms['test'](test_image)
    test_image_tensor.unsqueeze_(0)
    predict = model(test_image_tensor.to(device))
    result.append({'file': test_image_file, 'species': class_names[np.array(predict.max(1)[1][0])]})

In [55]:
import pandas as pd

In [56]:
submit = pd.DataFrame(data=result)

In [57]:
submit.head()

,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Fat Hen
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Sugar beet


In [58]:
len(submit)

794

In [59]:
submit.to_csv('submit-resnet34-50.csv', index=False)

In [62]:
#97.1% on test